In [ ]:
import numpy as np
import pandas as pd


hypso_comb = "remove"
sat_comb = "Hypso_filter"
sensor = "Landsat"
dir_path = "C:/Users/elise/OneDrive - NTNU/Documents/A.NTNU10/HYPSO data/Combined/trondheim_2024-09-14T09-42-48Z_trondheim_2024-04-29T09-44-07Z_/"
X_test_path = "C:/Users/elise/OneDrive - NTNU/Documents/A.NTNU10/HYPSO data/Combined/trondheim_2024-09-14T09-42-48Z_trondheim_2024-04-29T09-44-07Z_/X_test_remove.npy"
X_train_path = "C:/Users/elise/OneDrive - NTNU/Documents/A.NTNU10/HYPSO data/Combined/trondheim_2024-09-14T09-42-48Z_trondheim_2024-04-29T09-44-07Z_/X_train_remove.npy"
wavelength_file = "C:/Users/elise/Master/HYPSO/spectral_bands_HYPSO-1_v1.npz"

def map_hypso_to_satellite(hypso_comb, sat_comb, sensor, dir_path, X_train_path, X_test_path, wavelength_file):


    X_train = np.load(X_train_path)
    X_test = np.load(X_test_path)

    print(X_train.shape)
    print(X_test.shape)


    """
    kept_wavelengths = []
    dropped_bands = []
    select_wavelengths = True"""

    # Satellite bands (wavelengths in nanometers)
    all_wavelengths = np.load(wavelength_file)["arr_0"]
    all_wavelengths = all_wavelengths / 1000  # Convert nm to µm
    print(f"Number of bands: {len(all_wavelengths)}")
    print(f"Band 117: {all_wavelengths[116]} µm")
    bands_to_remove = [0,1,2,3,4,5, 106, 107, 108, 109, 119, 118, 117]
    band_nrs = np.arange(1, len(all_wavelengths) + 1)
    band_nrs = np.delete(band_nrs, bands_to_remove, axis=0)
    print(f"Band numbers: {band_nrs}")
    hypso_wavelengths = np.delete(all_wavelengths, bands_to_remove, axis=0)

    """if hypso_comb == "450-700":
        select_wavelengths = True
        kept_wavelengths = [450, 500, 550, 650, 700]
        print(f"Wavelengths to keep: {kept_wavelengths}")
    elif hypso_comb == "450-700_reflectance":
        select_wavelengths = True
        kept_wavelengths = [450, 500, 550, 650, 700]
        print(f"Wavelengths to keep: {kept_wavelengths}")
    elif hypso_comb == "all_filtered":
        select_wavelengths = False
        dropped_bands = [0,1,2,3,4,5, 119, 118, 117]
        print(f"Bands to drop: {dropped_bands}")
    elif hypso_comb == "all_filtered_reflectance":
        select_wavelengths = False
        dropped_bands = [0,1,2,3,4,5, 119, 118, 117]
        print(f"Bands to drop: {dropped_bands}")""""dimension_reduction filtered copy 2.ipynb"
    
    """

    if select_wavelengths:
        # Find the indices of the nearest wavelengths
        wavelength_indices = np.argmin(np.abs(all_wavelengths[:, None] - np.array(kept_wavelengths)[None, :]), axis=0)
        # Create the new array with selected wavelengths
        hypso_wavelengths = all_wavelengths[wavelength_indices]
        print(f"New number of bands: {len(hypso_wavelengths)}")

    else:
        hypso_wavelengths = np.delete(all_wavelengths, dropped_bands)
        print(f"New number of bands: {len(hypso_wavelengths)}")"""

    if sat_comb == "Hypso_filter":
        if sensor == "Landsat":
            satellite_wavelengths = np.array([(0.43+ 0.45)/2, (0.45+ 0.51)/2, (0.53+ 0.59)/2, (0.64+ 0.67)/2,(0.85+ 0.88)/2])  # µm
        elif sensor == "Sentinel":
            satellite_wavelengths = np.array([0.490,0.560,0.665,0.705,0.740,0.783,0.842,0.865])  # B2 spring, B3 winter, B4 spring, B5 winter, B6 summer, B7 summer, B8 summer, B8A summer
        """
    Maps Hypso hyperspectral data to satellite data (Sentinel or Landsat) based on wavelength proximity.

    Parameters:
        hypso_wavelengths (list or np.array): Wavelengths of Hypso bands.
        hypso_data (np.array): Hypso data with shape (n_samples, n_bands).
        satellite_wavelengths (list or np.array): Wavelengths of satellite bands.

    Returns:
        np.array: Mapped satellite data with shape (n_samples, len(satellite_wavelengths)).
    """
    mapped_X_train = []
    mapped_X_test = []
    for sat_wavelength in satellite_wavelengths:
        # Find Hypso bands within 10 nm of the satellite wavelength
        close_bands = np.where(np.abs(hypso_wavelengths - sat_wavelength) <= 0.01)[0] # 10 nm = 0.01 um
        
        
        if len(close_bands) > 0:
            # Average the values of the close bands
            print(f"Close bands for {sat_wavelength} µm: {band_nrs[close_bands]} - {hypso_wavelengths[close_bands]}")
            avg_values_train = np.mean(X_train[:, close_bands], axis=1)
            avg_values_test = np.mean(X_test[:, close_bands], axis=1)
        else:
            # If no bands are within the range, take the mean of the five closest bands
            closest_bands = np.argsort(np.abs(hypso_wavelengths - sat_wavelength))[:5]
            print(f"Out of range! Closest bands for {sat_wavelength} µm: {band_nrs[closest_bands]} - {hypso_wavelengths[closest_bands]}")  # +1 for 1-based indexing
            avg_values_train = np.mean(X_train[:, closest_bands], axis=1)
            avg_values_test = np.mean(X_test[:, closest_bands], axis=1)
        
        mapped_X_train.append(avg_values_train)
        mapped_X_test.append(avg_values_test)
    
    return np.array(mapped_X_train).T, np.array(mapped_X_test).T


# Map Hypso data to satellite inputs
mapped_X_train, mapped_X_test = map_hypso_to_satellite(hypso_comb, sat_comb, sensor, dir_path, X_train_path, X_test_path, wavelength_file)

print(mapped_X_train.shape)
print(mapped_X_test.shape)
print(mapped_X_train[:5])
print(mapped_X_test[:5])

# Save the mapped data
np.save(f"{dir_path}{hypso_comb}_X_train_{sensor}_{sat_comb}.npy", mapped_X_train)
np.save(f"{dir_path}{hypso_comb}_X_test_{sensor}_{sat_comb}.npy", mapped_X_test)